In [1]:
#Test for reading and writing from a dataframe within an optimisation algorithm
import numpy as np
import pandas as pd
import pyswarms as ps
from skopt import gp_minimize
from skopt.space import Real


In [ ]:


def auto_colour_func(col):
    ideal = np.array([40, 60, 20])
    j = ((col - ideal)**2).sum(axis = 1)
    return j

    
def robot_output(params):
    return params

def auto_input(params, df, iter_c):
    iter_s = params.shape[0]
    start_index = iter_c * iter_s
    values = auto_colour_func(params)
    df.values.reshape(96)[start_index:start_index+iter_s] = values
    df.to_csv('input.csv')
    return values
    
def user_input(params, df, iter_c):
    iter_s = params.shape[0]
    #handles input dataframe
    #for colour mixing 
    start_index = iter_c * iter_s
    #saves dataframe to file
    df.to_csv('input.csv')
    print('Input output values into input.csv')
    print("type 'yes' when done")
    inp = input()
    while inp != 'yes':
        inp = input()
    df2 = pd.read_csv('input.csv', index_col = [0], header = [0], dtype=np.double)
    values = df2.values.reshape(96)[start_index:start_index+iter_s]
    return values

class well_plate96:
    def __init__(self, function, iter_size, liquid_names):
        self.iteration_count = 0  # Initialize counter
        self.num_liquids = len(liquid_names)
        self.output = pd.DataFrame(data = np.zeros([8, 12*self.num_liquids]), index = np.arange(1, 9))
        self.output.columns = pd.MultiIndex.from_product([np.arange(1, 13), liquid_names])
        self.input = pd.DataFrame(data = np.zeros([8, 12]), index = np.arange(1, 9), columns = np.arange(1,13))
        self.iter_size = iter_size
        self.liquid_names = liquid_names
        #define which function you want, user input or colour mixing function
        self.function = function

    def __call__(self, params):
        #calculates start and end index
        start_index = (self.iteration_count * self.iter_size) * self.num_liquids
        end_index = start_index + self.iter_size*self.num_liquids

        #saves output parameter to csv
        self.output.values.reshape(96*self.num_liquids)[start_index:end_index] = params.reshape(12*self.num_liquids)
        self.output.to_csv('output.csv')

        #function to pippette goes here
        robot_output(params)

        #camera processing and colour extraction function goes here
        values = self.function(params, self.input, self.iteration_count)
        self.iteration_count += 1
        return values

In [3]:
model = well_plate96(auto_input, 12, ['blue', 'green', 'red'])

max_bound = 100 * np.ones(3)
min_bound = np.zeros(3)
bounds = (min_bound, max_bound)

#initialising swarm
options = {'c1': 0.3, 'c2': 0.5, 'w':0.1}

#Call instance of PSO with bounds argument
optimiser = ps.single.GlobalBestPSO(n_particles=12, dimensions=3, options=options, bounds=bounds)

#Perform optimization
cost, pos = optimiser.optimize(model, iters=8)


2025-02-28 13:43:50,839 - pyswarms.single.global_best - INFO - Optimize for 8 iters with {'c1': 0.3, 'c2': 0.5, 'w': 0.1}
pyswarms.single.global_best: 100%|██████████|8/8, best_cost=2.39
2025-02-28 13:43:50,903 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 2.386417510641984, best pos: [39.98955385 59.07966502 21.24068203]
